In [ ]:
import SimpleITK as sitk
import numpy as np

# Load iamge
def load_nifti_image(nifti_path):
    image = sitk.ReadImage(nifti_path)
    image = sitk.Cast(image, sitk.sitkFloat32)
    return image

# N4 bias field correction (May be slow)
def n4_bias_correction(image, mask=None):
    mask = sitk.OtsuThreshold(image) 
    corrector = sitk.N4BiasFieldCorrectionImageFilter()
    corrected_image = corrector.Execute(image, mask)
    return corrected_image

# resampling
def resample_image(image, new_spacing=(1, 1, 1), interpolator=sitk.sitkBSpline):
    original_spacing = image.GetSpacing()
    original_size = image.GetSize()
    new_size = [
        int(round(original_size[0] * (original_spacing[0] / new_spacing[0]))),
        int(round(original_size[1] * (original_spacing[1] / new_spacing[1]))),
        int(round(original_size[2] * (original_spacing[2] / new_spacing[2]))),
    ]
    
    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(new_spacing)
    resample.SetSize(new_size)
    resample.SetOutputDirection(image.GetDirection())
    resample.SetOutputOrigin(image.GetOrigin())
    resample.SetInterpolator(interpolator)
    resampled_image = resample.Execute(image)
    return resampled_image

# normalization
def normalize_image(image):
    array = sitk.GetArrayFromImage(image).astype(np.float32)
    array = (array - np.min(array)) / (np.max(array) - np.min(array))
    normalized_image = sitk.GetImageFromArray(array)
    normalized_image.CopyInformation(image)
    return normalized_image

# image preprocessing
def process_mri_image(input_nii_path, output_nii_path):
    image = load_nifti_image(input_nii_path)
    corrected_image = n4_bias_correction(image)
    resampled_image = resample_image(corrected_image, new_spacing=[1.0, 1.0, 1.0])
    normalized_image = normalize_image(resampled_image)
    sitk.WriteImage(normalized_image, output_nii_path)

# example usage
input_path = "File path"  
output_path = "File path"  

process_mri_image(input_path, output_path)
